In [ ]:
import sys
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import csv
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
import time

chrome_options = Options()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('log-level=2')sys
driver = webdriver.Chrome('./chromedriver', options=chrome_options)
katakunci = input('Masukkan kata kunci : ')

def search(katakunci):
    links = []
    
    print('mencari semua product dengan kata kunci ' + katakunci)
    url = 'https://shopee.co.id/search?keyword=' + katakunci
    try:
        driver.get(url)
        time.sleep(5)
        driver.execute_script('window.scrollTo(0, 1500);')
        time.sleep(5)
        driver.execute_script('window.scrollTo(0, 2500);')
        time.sleep(5)
        soup_a = BeautifulSoup(driver.page_source, 'html.parser')
        products = soup_a.find('div', class_='row shopee-search-item-result__items')
        for link in products.find_all('a'):
            links.append(link.get('href'))
            print(link.get('href'))
    except TimeoutException:
        print('failed to get links with query ')
    
    return links

product_urls = search(katakunci)


In [ ]:
def get_product(produt_url):
    try:
        url = 'https://shopee.co.id' + produt_url
        driver.get(url)
        time.sleep(10)
        driver.execute_script('window.scrollTo(0, 1500);')
        time.sleep(10)
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CLASS_NAME, 'YPqix5'))) ## Class dapat berubah2
        soup_b = BeautifulSoup(driver.page_source, 'html.parser')
        title = soup_b.find('div', class_='YPqix5').text
        price = soup_b.find('div', class_='X0xUb5').text
        try:
            image = soup_b.find('div', class_='_3VFGEz wdZ+2f')['style'] ## Class dapat berubah2
            imgurl = re.findall('url\((.*?)\)', image)
        except:
            imgurl = 'none'
        desc = soup_b.find('div', class_='sJarux').text
        print('Scraping ' + title)

        # kita simpan hasil scraping ke file sresult.csv
        with open('sresult.csv','a', encoding='utf-8',newline='') as f:
            writer=csv.writer(f)
            writer.writerow([title,price,url,desc,imgurl])

    except TimeoutException:
        print('cant open the link')

# products_urls = search(katakunci)

for product_url in product_urls:
    get_product(product_url)

driver.quit()